In [24]:
import requests
import json
from bs4 import BeautifulSoup
from datetime import datetime
from transformers import pipeline

In [25]:
def scrape_reddit_topics(search_query):
    # Use search endpoint with query
    url = f"https://www.reddit.com/search.json?q={search_query}&sort=relevance"
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.3'
    }
    try:
        # Get search results
        response = requests.get(url, headers=headers)
        data = response.json()
        
        posts_with_comments = []
        
        for post in data['data']['children']:
            post_data = post['data']
            
            # Get comments for this post
            comments_url = f"https://www.reddit.com{post_data['permalink']}.json"
            comments_response = requests.get(comments_url, headers=headers)
            comments_data = comments_response.json()
            
            # Extract comments
            comments = []
            if len(comments_data) > 1:  # Check if there are comments
                for comment in comments_data[1]['data']['children']:
                    if 'body' in comment['data']:
                        comments.append({
                            'author': comment['data'].get('author', '[deleted]'),
                            'body': comment['data']['body'],
                            'score': comment['data'].get('score', 0),
                            'created_utc': datetime.fromtimestamp(comment['data']['created_utc']).isoformat()
                        })
            
            # Combine post and comments data
            posts_with_comments.append({
                'title': post_data['title'],
                'author': post_data['author'],
                'score': post_data['score'],
                'url': post_data['url'],
                'created_utc': datetime.fromtimestamp(post_data['created_utc']).isoformat(),
                'num_comments': post_data['num_comments'],
                'selftext': post_data.get('selftext', ''),
                'comments': comments
            })
        
        # Save to JSON file
            
        return posts_with_comments
        
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        return None

In [26]:
#webscraping particular subreddits
print("Hi! Welcome to the Reddit Web Scraper!")
subreddit = input("Please enter the subreddit you would like to scrape: ")
posts = scrape_reddit_topics(subreddit)

Hi! Welcome to the Reddit Web Scraper!


In [27]:
#Scraped data is in JSON format


In [28]:
def extract_posts_and_comments(json_obj):
    texts = []
    for entry in json_obj:
        # Extract post content (selftext)
        if 'selftext' in entry and entry['selftext']:
            texts.append(entry['selftext'])
        
        # Extract comments
        if 'comments' in entry:
            for comment in entry['comments']:
                if 'body' in comment:
                    texts.append(comment['body'])
    return texts

posts_and_comments = extract_posts_and_comments(posts)



In [29]:
#NLP stuff 
classifier = pipeline('sentiment-analysis', model='allenai/longformer-base-4096')

results=[]
for text in posts_and_comments:
    result = classifier(text)
    results.append(result)

print(results)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tf_keras/src/initializers/initializers.py:121: UserWarning: The initializer TruncatedNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
Some layers from the model checkpoint at allenai/longformer-base-4096 were not used when initializing TFLongformerForSequenceClassification: ['lm_head']
- This IS expected if you are initializing TFLongformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFLongformerForSequenceClassification from the checkpoint of a model that you expect to be exactly iden

[[{'label': 'LABEL_0', 'score': 0.5849762558937073}], [{'label': 'LABEL_0', 'score': 0.5890578031539917}], [{'label': 'LABEL_0', 'score': 0.5833932757377625}], [{'label': 'LABEL_0', 'score': 0.5847958922386169}], [{'label': 'LABEL_0', 'score': 0.5859900116920471}], [{'label': 'LABEL_0', 'score': 0.5869458317756653}], [{'label': 'LABEL_0', 'score': 0.5860555171966553}], [{'label': 'LABEL_0', 'score': 0.5868340134620667}], [{'label': 'LABEL_0', 'score': 0.5838847160339355}], [{'label': 'LABEL_0', 'score': 0.5884658098220825}], [{'label': 'LABEL_0', 'score': 0.5851649641990662}], [{'label': 'LABEL_0', 'score': 0.5828875303268433}], [{'label': 'LABEL_0', 'score': 0.5830407738685608}], [{'label': 'LABEL_0', 'score': 0.5822809934616089}], [{'label': 'LABEL_0', 'score': 0.5884075164794922}], [{'label': 'LABEL_0', 'score': 0.582288384437561}], [{'label': 'LABEL_0', 'score': 0.5892163515090942}], [{'label': 'LABEL_0', 'score': 0.5882212519645691}], [{'label': 'LABEL_0', 'score': 0.5852292180061